In [1]:
from llm.llm.chatgpt import ChatGPT
from setup_db import create_chroma_db, connect_to_db, execute_query

In [2]:
llm = ChatGPT()

In [3]:
collection_chromadb = 'test_chroma_db'
persist_directory = 'data/test_chroma_db'
vector_store = create_chroma_db(collection_chromadb, persist_directory)

In [4]:
def similarity_search(query, k):
    return [f"'{document.page_content}'" for document in vector_store.similarity_search(query, k)]

In [5]:
from llm_test import text2sql, replace_category_query, CoT_reasoning

In [6]:
prompt = "What is the revenue and the profit of the companies with stock code TCB and VCB in first quarter of 2024"
code_generated = text2sql(llm, prompt)
code_generated

[{'language': 'sql',
  'code': "SELECT * \nFROM financial_statement fs \nWHERE ticker IN ('TCB', 'VCB') \nAND yearReport = 2024 \nAND lengthReport = 1 \nAND (category = 'Revenue' OR category = 'Profit');\n"}]

In [7]:
for code in code_generated:
    if code['language'] == 'sql':
        code['code'] = replace_category_query(code['code'], similarity_search, top_k=5)
        # vector_store.execute_query(query)

In [12]:
test = "category = 'HYY'"
replace_category_query(test, similarity_search, top_k=5)

"category = 'HYY'"

In [18]:

# import re

# def top_k(text):
#     # Placeholder for top_k function, replace this with the actual implementation
#     return [f"top_k({text})"]

# def replace_category_query(sql_query):
#     # Pattern to match `category = xx`
#     pattern_equal = re.compile(r"category\s*=\s*'?(\w+)'?")
#     # Pattern to match `category IN (xx, yy)` or `category IN ('xx', 'yy')`
#     pattern_in = re.compile(r"category\s+IN\s*\(([^)]+)\)")
    
#     # Check for `category = xx`
#     if pattern_equal.search(sql_query):
#         sql_query = pattern_equal.sub(lambda match: f"category IN ({', '.join(top_k(match.group(1)))})", sql_query)
    
#     # Check for `category IN (xx, yy)`
#     elif pattern_in.search(sql_query):
#         sql_query = pattern_in.sub(lambda match: f"category IN ({', '.join([item for x in match.group(1).split(',') for item in top_k(x.strip())])})", sql_query)

#     return sql_query

# # Example usage:
# sql_query_1 = "category = 'xx'"
# sql_query_2 = "category IN ('xx', yy)"

# print(replace_category_query(sql_query_1))  # Output for equal size
# print(replace_category_query(sql_query_2))  # Output for IN condition

category IN (top_k(xx))
category IN (top_k('xx'), top_k(yy))


In [8]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
table_name = 'financial_statement'
port = '5433'
host = 'localhost'

In [9]:
code_generated

[{'language': 'sql',
  'code': "SELECT * \nFROM financial_statement fs \nWHERE ticker IN ('TCB', 'VCB') \nAND yearReport = 2024 \nAND lengthReport = 1 \nAND (category = 'Revenue' OR category = 'Profit');\n"}]

In [10]:
conn = connect_to_db(db_name, user, password, host, port)
result = execute_query(code_generated[0]['code'], conn=conn)
result

Connecting to database test_db, postgres...


[]

In [11]:
answer = CoT_reasoning(llm, prompt, result)
print(answer)

To determine the revenue and the profit of the companies with stock codes TCB and VCB for the first quarter of 2024 based on the given financial statements, we will follow these systematic steps:

### Step 1: Identify Available Data

1. **Review the Financial Statements**: Since the provided financial statement data is empty (`[]`), we have no explicit information regarding the revenues or profits for TCB and VCB for the specified quarter.

2. **Understand Financial Statement Components**: Typically, a financial statement includes:
   - **Revenue**: This represents the total income generated from business operations before any expenses are deducted.
   - **Net Profit**: This is the total earnings after all expenses, taxes, and costs have been deducted from revenue.

### Step 2: Locate Relevant Financial Information

1. **Source Data**: Since we do not have the data directly within the provided result, we need to obtain this information from reliable financial databases, company reports

category = xx

category IN (xx, yy)